# Naive implementation

In [2]:
from copy import deepcopy
from itertools import permutations, product

class Solver:

    def __init__(
            self,
            target_number: int,
            numbers: list[int],
            accuracy: float = 0.00000001, # print(3 / (1 - 2 / 3)) >>> 8.999999999999998
        ) -> None:

        # Initialize the Solver object with target number, numbers to operate on, and accuracy
        self.target_number = target_number
        self.numbers = numbers

        # Accuracy to determine how close a solution needs to be to the target number
        self.accuracy = accuracy

        # Create a list of all unique permutations of the given numbers
        self.numbers_list = [list(nums) for nums in set(permutations(numbers))]
        self.iter_num = len(numbers) - 1

        # Dictionary of basic arithmetic operators
        self.operators = {
            '+': lambda x, y: x + y,
            '-': lambda x, y: x - y,
            '*': lambda x, y: x * y,
            '/': lambda x, y: x / y,
        }

        # List of all possible combinations of operators and their calculation orders
        self.operators_with_calc_order = self._list_operator_calc_order_pair()

    def _list_operator_calc_order_pair(self) -> list[dict[int, str]]:
        # Generate all possible calculation orders for given number of operands
        calc_orders = list(permutations([num for num in range(self.iter_num)]))

        operators_with_calc_order = []

        # Generate all possible combinations of operators and calculation orders
        for op_combination in product(self.operators.keys(), repeat=self.iter_num):
            for calc_order in calc_orders:
                operators_with_calc_order.append({order: op for order, op in zip(calc_order, op_combination)})

        return operators_with_calc_order

    def calculate(self, numbers: list[int], op_order: dict[int, str]) -> float:
        # Deep copy the numbers to avoid modifying the original list
        nums = deepcopy(numbers)
        idxs_used = []

        # Iterate through each operator and perform the corresponding calculation
        for i in range(self.iter_num):
            tmp = list(op_order).index(i)
            op = op_order[i]

            # Check for division by zero
            if op == '/' and nums[tmp + 1] == 0:
                return None

            # Perform the operation and update the result in the list
            res = self.operators[op](nums[tmp], nums[tmp + 1])

            # Track the indices used in the calculation
            if tmp not in idxs_used:
                idxs_used.append(tmp)
            if tmp + 1 not in idxs_used:
                idxs_used.append(tmp + 1)

            # Update the list with the result of the calculation
            for idx in idxs_used:
                nums[idx] = res

        return nums[0]

    def solve(self) -> list[dict]:
        # List to store solutions
        solutions = []

        # Iterate through each permutation of numbers and each combination of operators and their orders
        for numbers in self.numbers_list:
            for op_order in self.operators_with_calc_order:
                # Calculate the result
                res = self.calculate(numbers, op_order)

                # Check if the result is within the desired accuracy of the target number
                if res is not None and abs(res - self.target_number) <= self.accuracy:
                    # If so, store the solution
                    solution = {
                        "target_number": self.target_number,
                        "numbers": numbers,
                        "op_order": op_order
                    }

                    solutions.append(solution)

        # If no solutions were found, print a message
        if not solutions:
            print('No Solutions')

        return solutions

    def visualise(self, solution: dict) -> None:
        target_num = deepcopy(solution['target_number'])
        nums = deepcopy(solution['numbers'])
        op_order = deepcopy(solution['op_order'])
        idxs_used = []

        for i in range(self.iter_num):
            tmp = list(op_order).index(i)
            op = op_order[i]

            if i != self.iter_num - 1:
                res = f'({nums[tmp]}{op}{nums[tmp + 1]})'
            elif i == self.iter_num - 1:
                res = f'{nums[tmp]}{op}{nums[tmp + 1]}'

            if tmp not in idxs_used:
                idxs_used.append(tmp)
            if tmp + 1 not in idxs_used:
                idxs_used.append(tmp + 1)

            for idx in idxs_used:
                nums[idx] = res

        print(nums[0])

In [3]:
numbers = [3, 3, 8, 8]
target_number = 24

s = Solver(target_number, numbers)
solutions = s.solve()
solutions

[{'target_number': 24,
  'numbers': [8, 3, 8, 3],
  'op_order': {2: '/', 1: '-', 0: '/'}}]

In [4]:
sol = solutions[0]
sol

{'target_number': 24,
 'numbers': [8, 3, 8, 3],
 'op_order': {2: '/', 1: '-', 0: '/'}}

In [5]:
s.visualise(sol)

8/(3-(8/3))


In [6]:
eval("8/(3-(8/3))")

23.99999999999999

# Advanced Method

https://pydocument.hatenablog.com/entry/2023/03/25/140055

- 逆ポーランド記法
- スタック

ー＞ tree を処理

https://qiita.com/Yuya-Shimizu/items/86a2246306bad769462d

最終的には木構造を深さ優先探索したい

In [ ]:
def evaluate(expression):
    stack = []
    for element in expression.split():
        if element.isdigit():
            stack.append(int(element))
        else:
            num2 = stack.pop()
            num1 = stack.pop()
            if element == '+':
                stack.append(num1 + num2)
            elif element == '-':
                stack.append(num1 - num2)
            elif element == '*':
                stack.append(num1 * num2)
            elif element == '/':
                stack.append(num1 / num2)
    return stack.pop()

1. まずは演算子と数字を一種類ずつ用いて考えうる逆ポーランド記法の数式の全てを書き出す

2. 使用できる演算子全ての組み合わせを書き下す

3. 2 で書き下した各表現について，使用できる数字で全ての組み合わせを作る

1. 演算子の前には２つの数字が積まれていないといけない

n 個の数字と，n - 1 個の演算子

これこそ木構造かも

In [ ]:
nums = [1, 1, 1, 1]
# ops = ['・', '・', '・']
ops = [-1, -1, -1]

# 前から足して常に > 0 になるように並べる

# tree で任意の階層の child node の生成の際に使えそう（再帰）
eval("print('Hello world')")